In [ ]:
import tweepy
import json
from pymongo import MongoClient
from datetime import datetime
import time
import sys

import emoji
import re

In [ ]:
from twittercredentials import consumer_key, consumer_secret, access_token, access_token_secret

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret )
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
if (not api):
    print('Can\'t authenticate')
    print('failed cosumeer id ----------: ', consumer_key )

In [ ]:
# entities based on keywords chosen from clustering
query = "scotrail"  #test query at the moment to test

In [ ]:
count = 100 # number of tweets to grab in one go

last_id = -1    # used to update last id to gather tweets from different users
new_tweets = True   # used to go round in loop
counter = -1    # counter to update times we have gathered data

In [ ]:
storing_tweets = []

In [ ]:
while new_tweets:
        counter+=1  # adds one to the counter everytime we try and search for tweets

        try:
            if(counter < 180):  #this is so we do not go over 180 requests as this is Rest API limit
                new_tweets = api.search(q=query, count=count, lang="en", tweet_mode='extended', max_id=str(last_id - 1))
         
                if not new_tweets:  # makes sure there is tweets to look at
                    break
          
                for t in new_tweets:
                   
                    t = json.dumps(t._json) #   Rest API has to load json differently
                    tweet = json.loads(t)

                    retweet = False

                    if tweet['retweeted'] or tweet['full_text'].startswith('RT') == True:
                        retweet = True

                    if tweet['truncated']:
                        text = tweet['extended_tweet']['full_text']
                    else:
                        text = tweet['full_text']

                    if (retweet == False) and (text not in storing_tweets): #no point adding same text to list, will encourage bot tweets etc
                        storing_tweets.append(text)
                     
                last_id = new_tweets[-1].id
            else:
                print("15 minute delay")
                break   #just so can test data atm
                counter = 0 # sets counter back to zero
                time.sleep(15*60)   # sleeps for 15 mins to comply with regulations
                
        except tweepy.TweepError as e:
            break
            

In [ ]:
print(storing_tweets)

## VADER Sentiment Analysis with twitter, testing obviously

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Example code:

f = open("tweet_scores.txt", "x", encoding="utf-8")

for twitter_post in storing_tweets:
  vs = analyzer.polarity_scores(twitter_post)
  compound = vs['compound']
  sentiment_score = compound * 100
  print("The post '" + twitter_post + "' has a sentiment score of: " + (str(sentiment_score)) + "%")
  
  if (compound >= 0.05):
    print("The post was POSITIVE")
    f.write("\n" + twitter_post + ": RESULT = POSITIVE")
  elif (compound <= -0.05):
    print("The post was NEGATIVE")
    f.write("\n" + twitter_post + ": RESULT = NEGATIVE")
  else:
    print("The post was NEUTRAL")
    f.write("\n" + twitter_post + ": RESULT = NEUTRAL")
  print("{:-<65} {}".format(twitter_post, str(vs)))

f.close()